# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [24]:
data.head()

data.loc[data['CustomerID'] == 33]

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
527,33,Lindsay,Santana,1899667,432,Beer - Original Organic Lager,1
1017,33,Lindsay,Santana,6509275,417,"Lamb - Pieces, Diced",1
3012,33,Lindsay,Santana,5638266,214,French Pastry - Mini Chocolate,1
3476,33,Lindsay,Santana,4280329,284,Beer - Rickards Red,1
3795,33,Lindsay,Santana,5869069,342,Mangoes,1
4118,33,Lindsay,Santana,1888258,53,Cassis,1
6104,33,Lindsay,Santana,1465031,320,Yogurt - French Vanilla,1
8040,33,Lindsay,Santana,5150162,309,Muffin Batt - Choc Chk,1
8624,33,Lindsay,Santana,6214157,199,Pork - Hock And Feet Attached,1
8781,33,Lindsay,Santana,2958865,91,General Purpose Trigger,1


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [55]:
grouped = data.groupby(['CustomerID', 'ProductID'])[["Quantity"]].sum().reset_index()

In [56]:
grouped.head(100)

,CustomerID,ProductID,Quantity
0,33,23,1
1,33,51,1
2,33,53,1
3,33,67,1
4,33,70,1
5,33,77,1
6,33,91,1
7,33,110,1
8,33,138,1
9,33,142,1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [63]:
product_matrix = grouped.pivot_table(index='ProductID', columns='CustomerID', values='Quantity', fill_value=0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [66]:
customer_dist = squareform(pdist(product_matrix.T, metric='euclidean'))

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [77]:
customer_dist

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.